In [18]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_openai import ChatOpenAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_classic.chains import RetrievalQA
from langchain_chroma import Chroma
import dotenv
import os


In [12]:
dotenv.load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")

In [ ]:
CAMINHO_PDF = "documentos/regras_do_Jogo_2023_24.pdf" 

loader = PyPDFLoader(CAMINHO_PDF)
documents = loader.load()

len(documents)

232

In [5]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)

chunks = text_splitter.split_documents(documents)

len(chunks)

654

In [ ]:
embeddings = GoogleGenerativeAIEmbeddings(
    model="models/text-embedding-004",
    google_api_key=api_key
)

NOME_PASTA_DB = "./chroma_regras_futebol"

if os.path.exists(NOME_PASTA_DB):
    print("✅ Banco de dados encontrado localmente. Carregando...")
    vectorstore = Chroma(
        persist_directory=NOME_PASTA_DB,
        embedding_function=embeddings
    )
else:
    print("❌ Banco não encontrado. Criando e gerando embeddings")
    vectorstore = Chroma.from_documents(
        documents=chunks,
        embedding=embeddings,
        persist_directory=NOME_PASTA_DB
    )

✅ Banco de dados encontrado localmente. Carregando...


In [17]:
# Cria o retriever
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)

In [21]:
llm = ChatGoogleGenerativeAI(
    model="gemini-flash-latest", 
    temperature=0.2, 
    google_api_key=api_key
)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

In [22]:
pergunta = "Explique a regra de mão na bola em momentos de ataque no futebol."

resposta = qa_chain(pergunta)

print("Pergunta:")
print(pergunta)

print("\nResposta do Agente:")
print(resposta["result"])

print("\nTrechos utilizados como contexto:\n")
for i, doc in enumerate(resposta["source_documents"], start=1):
    print(f"--- Trecho {i} ---")
    print(f"Fonte: {doc.metadata.get('source', 'Documento desconhecido')}")
    print(f"Página: {doc.metadata.get('page', 'N/A')}\n")
    print("Conteúdo:")
    print(doc.page_content)
    print("\n")

Pergunta:
Explique a regra de mão na bola em momentos de ataque no futebol.

Resposta do Agente:
Com base nas informações fornecidas, a regra de mão na bola em momentos de ataque é clara em relação à consequência imediata do toque:

Uma infração é marcada se um gol for marcado (ou se a jogada que resulta no gol for beneficiada) **imediatamente depois de a bola tocar na mão ou no braço**, mesmo que o toque tenha ocorrido de maneira acidental.

O contexto também define que o limite superior do braço, para fins de determinação da infração, se alinha com o ponto inferior da axila. No entanto, nem todos os contatos da mão ou do braço de um jogador com a bola constituem uma infração, exceto quando o toque leva a uma vantagem imediata de ataque, conforme descrito acima.

Trechos utilizados como contexto:

--- Trecho 1 ---
Fonte: documentos/regras_do_Jogo_2023_24.pdf
Página: 104

Conteúdo:
de equipe;
 • o goleiro a receber diretamente de um arremesso lateral executado por um   
   companheiro 